# Embed gsm8k using modal parallelization


In [1]:
%env HF_USE_HF_TRANSFER 1
%pip install datasets hf_transfer umap-learn -q

env: HF_USE_HF_TRANSFER=1
Note: you may need to restart the kernel to use updated packages.


In [3]:
repo_id = "mistobaan/gsm8k-train-nomic-text-v1.5"

In [5]:
import os
if not os.path.exists(repo_id.split('/')[-1]):
    !git clone https://huggingface.co/datasets/{repo_id} --depth 1

In [6]:
import multiprocessing
import umap

cpu_count = max(multiprocessing.cpu_count() - 1, 1)
umap_model = umap.UMAP(
    n_components=2, random_state=2023, verbose=True, n_jobs=cpu_count
)

/opt/homebrew/anaconda3/envs/mlx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import datasets

ds = datasets.load_from_disk(f'./{repo_id.split("/")[-1]}')
df = ds.to_pandas()

In [10]:
embeddings = ds["question_embedding"]

In [11]:
df[["x", "y"]] = umap_model.fit_transform(X=embeddings)

/opt/homebrew/anaconda3/envs/mlx/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


UMAP(n_jobs=1, random_state=2023, verbose=True)
Mon Apr 22 17:55:55 2024 Construct fuzzy simplicial set
Mon Apr 22 17:55:55 2024 Finding Nearest Neighbors
Mon Apr 22 17:55:55 2024 Building RP forest with 9 trees
Mon Apr 22 17:55:56 2024 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	Stopping threshold met -- exiting after 6 iterations
Mon Apr 22 17:56:01 2024 Finished Nearest Neighbor Search
Mon Apr 22 17:56:02 2024 Construct embedding


Epochs completed:   2%| ▏          11/500 [00:00]

	completed  0  /  500 epochs


Epochs completed:  12%| █▏         59/500 [00:01]

...

	completed  400  /  500 epochs


Epochs completed:  92%| █████████▏ 459/500 [00:06]

	completed  450  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:07]

Mon Apr 22 17:56:09 2024 Finished embedding


In [12]:
import pandas as pd


def normalize(s: pd.Series, upper_bound=100):  # lower bound is 1
    return upper_bound * ((s - s.min()) / (s.max() - s.min()))


def normalize_x_and_y(data: pd.DataFrame):
    temp = data.copy().dropna()
    temp["x"] = normalize(temp.x)
    temp["y"] = normalize(temp.y)
    NUM_DECIMALS = 6  # aka no rounding
    temp["x"] = temp["x"].round(NUM_DECIMALS)
    temp["y"] = temp["y"].round(NUM_DECIMALS)
    return temp

In [13]:
normalized_df = normalize_x_and_y(df)

In [14]:
normalized_df[["x", "y", "question", "answer"]].to_csv(
    "./dataset-quadfeather-ready.csv", index=False
)

In [16]:
!/opt/homebrew/anaconda3/bin/quadfeather --files dataset-quadfeather-ready.csv --tile_size 10000 --destination ./deepscatter-tiles